In [1]:
import gc
import logging
logging.disable(logging.WARNING)
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras import mixed_precision
from tensorflow.keras.utils import plot_model
import tensorflow_probability as tfp

mixed_precision.set_global_policy('mixed_float16')
print(tf.__version__)

2.6.3


In [ ]:
# Load data
df = pd.read_parquet('E:/train_low_mem.parquet')
df.drop(['row_id'], inplace=True, axis=1)
investment_id = df.pop("investment_id")
y = df.pop("target")

investment_ids = list(investment_id.unique())
investment_id_lookup_layer = IntegerLookup(oov_token=-1, output_mode='int')
investment_id_lookup_layer.adapt(np.array(investment_ids))
vocab = investment_id_lookup_layer.get_vocabulary(include_special_tokens=True)
vocab_size = investment_id_lookup_layer.vocabulary_size()
print(max(investment_ids))
print(vocab_size)

In [ ]:
def preprocess(x, y):
    # TODO
    return x, y

def make_dataset(feature, investment_id, y, batch_size=2048, mode='train'):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train": ds = ds.shuffle(8192, seed=69)  # only shuffle when training
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds


# Pearson correlation coefficient loss and metrics
def pearson_corr(y_true, y_pred, axis=-1):
    return tfp.stats.correlation(tf.squeeze(y_pred), tf.squeeze(y_true), sample_axis=axis, event_axis=None)

def pearson_corr_loss(y_true, y_pred, axis=-1):
    return (1 - pearson_corr(y_true, y_pred, axis=axis)) + 2 * tf.keras.metrics.mean_squared_error(y_true, y_pred)

In [ ]:
# Model
def build_model():
    investment_id_inputs = Input((1, ), name='inv_id_in', dtype=tf.uint16)
    features_inputs = Input((300, ), name='feature_in', dtype=tf.float16)

    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = Embedding(vocab_size, 64, input_length=1)(investment_id_x)
    investment_id_x = Reshape((-1, ))(investment_id_x)
    investment_id_x = Dense(64, activation='swish')(investment_id_x)
    # investment_id_x = Dropout(0.2)(investment_id_x)
    investment_id_x = Dense(64, activation='swish')(investment_id_x)
    # investment_id_x = Dropout(0.2)(investment_id_x)
    investment_id_x = Dense(64, activation='swish')(investment_id_x)
    # investment_id_x = Dropout(0.2)(investment_id_x)

    feature_x = GaussianNoise(0.01)(features_inputs)
    feature_x = Dense(256, activation='swish')(feature_x)
    # feature_x = Dropout(0.2)(feature_x)
    feature_x = Dense(256, activation='swish')(feature_x)
    # feature_x = Dropout(0.2)(feature_x)
    feature_x = Dense(256, activation='swish')(feature_x)
    # feature_x = Dropout(0.2)(feature_x)

    x = Concatenate(axis=1)([investment_id_x, feature_x])
    x = Dense(512, activation='swish', kernel_regularizer="l2")(x)
    # x = LayerNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(128, activation='swish', kernel_regularizer="l2")(x)
    # x = LayerNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(32, activation='swish', kernel_regularizer="l2")(x)
    # x = LayerNormalization()(x)
    x = Dropout(0.2)(x)
    xOut = Dense(1, name='output')(x)

    model = Model(inputs=[investment_id_inputs, features_inputs], outputs=[xOut])
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=7e-4), metrics=[pearson_corr, 'mse'])
    return model

In [ ]:
model = build_model()
model.summary()
plot_model(model, to_file='model.png', show_shapes=True)

In [7]:
from typing import Tuple
import numpy as np

class GroupTimeSeriesSplit:
    """
    Custom class to create a Group Time Series Split. We ensure
    that the time id values that are in the testing data are not a part
    of the training data & the splits are temporal
    """
    def __init__(self, n_folds: int, holdout_size: int, groups: str) -> None:
        self.n_folds = n_folds
        self.holdout_size = holdout_size
        self.groups = groups

    def split(self, X) -> Tuple[np.array, np.array]:
        # Take the group column and get the unique values
        unique_time_ids = np.unique(self.groups.values)

        # Split the time ids into the length of the holdout size
        # and reverse so we work backwards in time. Also, makes
        # it easier to get the correct time_id values per
        # split
        array_split_time_ids = np.array_split(
            unique_time_ids, len(unique_time_ids) // self.holdout_size
        )[::-1]

        # Get the first n_folds values
        array_split_time_ids = array_split_time_ids[:self.n_folds]

        for time_ids in array_split_time_ids:
            # Get test index - time id values that are in the time_ids
            test_condition = X['time_id'].isin(time_ids)
            test_index = X.loc[test_condition].index

            # Get train index - The train index will be the time
            # id values right up until the minimum value in the test
            # data - we can also add a gap to this step by
            # time id < (min - gap)
            train_condition = X['time_id'] < (np.min(time_ids))
            train_index = X.loc[train_condition].index

            yield train_index, test_index

In [ ]:
import scipy
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=69)
# group_time_series_split = GroupTimeSeriesSplit(n_folds=6, holdout_size=200, groups=df['time_id'])
models = []

for index, (train_indices, valid_indices) in enumerate(kfold.split(df, investment_id)):
    print(f'Model {index}')
    X_train, X_val = df.iloc[train_indices].loc[:, df.columns != 'time_id'], df.iloc[valid_indices].loc[:, df.columns != 'time_id']
    investment_id_train = investment_id[train_indices]
    y_train, y_val = y.iloc[train_indices], y.iloc[valid_indices]
    investment_id_val = investment_id[valid_indices]
    train_ds = make_dataset(X_train, investment_id_train, y_train)
    valid_ds = make_dataset(X_val, investment_id_val, y_val, mode="valid")
    model = build_model()
    callbacks = [tf.keras.callbacks.ModelCheckpoint(f"model_{index}", monitor='val_mse', save_best_only=True, mode='min'),
                 tf.keras.callbacks.EarlyStopping(patience=13, monitor='val_mse', verbose=1, restore_best_weights=True, mode='min'),
                 tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=6, verbose=1, monitor='val_mse', mode='min'),
                 tf.keras.callbacks.TerminateOnNaN()]
    history = model.fit(train_ds, epochs=100, validation_data=valid_ds, callbacks=callbacks)
    models.append(load_model(f"model_{index}", custom_objects={'pearson_corr': pearson_corr, 'pearson_corr_loss': pearson_corr_loss}))

    pearson_score = scipy.stats.pearsonr(model.predict(valid_ds).ravel(), y_val.values)[0]
    print('Pearson:', pearson_score)
    del investment_id_train
    del investment_id_val
    del X_train
    del X_val
    del y_train
    del y_val
    del train_ds
    del valid_ds
    del model
    gc.collect()

In [12]:
import scipy
X_train, y_train = df.loc[:, df.columns != 'time_id'], y
investment_id_train = investment_id
# train_ds = make_dataset(X_train, investment_id_train, y_train)
model = build_model()
callbacks = [tf.keras.callbacks.ModelCheckpoint(f"model_comb", monitor='val_mse', save_best_only=True, mode='min'),
             tf.keras.callbacks.EarlyStopping(patience=13, monitor='val_mse', verbose=1, restore_best_weights=True, mode='min'),
             tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=6, verbose=1, monitor='val_mse', mode='min'),
             tf.keras.callbacks.TerminateOnNaN()]
history = model.fit({'inv_id_in': investment_id_train, 'feature_in': X_train}, y_train, batch_size=2048, epochs=100, validation_split=0.2, callbacks=callbacks)

Epoch 1/100
1228/1228 [==============================] - 11s 7ms/step - loss: 0.9932 - pearson_corr: 0.1395 - mse: 0.8369 - val_loss: 0.8095 - val_pearson_corr: 0.1212 - val_mse: 0.8062
Epoch 2/100
1228/1228 [==============================] - 8s 7ms/step - loss: 0.8351 - pearson_corr: 0.1595 - mse: 0.8319 - val_loss: 0.8114 - val_pearson_corr: 0.1109 - val_mse: 0.8082
Epoch 3/100
1228/1228 [==============================] - 7s 5ms/step - loss: 0.8325 - pearson_corr: 0.1711 - mse: 0.8288 - val_loss: 0.8133 - val_pearson_corr: 0.1189 - val_mse: 0.8097
Epoch 4/100
1228/1228 [==============================] - 6s 5ms/step - loss: 0.8291 - pearson_corr: 0.1825 - mse: 0.8254 - val_loss: 0.8097 - val_pearson_corr: 0.1304 - val_mse: 0.8064
Epoch 5/100
1228/1228 [==============================] - 6s 5ms/step - loss: 0.8261 - pearson_corr: 0.1930 - mse: 0.8219 - val_loss: 0.8198 - val_pearson_corr: 0.1173 - val_mse: 0.8151
Epoch 6/100
1228/1228 [==============================] - 6s 5ms/step - los

In [9]:
from ubiquantEmulator import TimeSeriesAPI

def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0

def make_test_dataset(feature, investment_id, batch_size=2048):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds


def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)  # ensemble

models = [load_model(f"model_{index}", custom_objects={'pearson_corr': pearson_corr, 'pearson_corr_loss': pearson_corr_loss}) for index in range(6)]
models.pop(3)
# models = [load_model(f"model_comb", custom_objects={'pearson_corr': pearson_corr, 'pearson_corr_loss': pearson_corr_loss})]
# models += [load_model(f"1/model_{index}", custom_objects={'pearson_corr': pearson_corr, 'pearson_corr_loss': pearson_corr_loss}) for index in range(6)]
# test = pd.read_parquet('E:/train_low_mem.parquet')
# time_ids = df['time_id'].unique()
# test = test.loc[test['time_id'].isin(time_ids)]  # take last 200 time id rows from df
test = pd.read_csv('kaggle/input/ubiquant-market-prediction/train.csv', engine='pyarrow')
iter_test = TimeSeriesAPI(test)
for n, (test_df, sample_prediction_df) in enumerate(iter_test):
    ds = make_test_dataset(test_df.loc[:, 'f_0':'f_299'], test_df["investment_id"])
    sample_prediction_df['target'] = inference(models, ds)
    iter_test.predict(sample_prediction_df)
print(sample_prediction_df)
print(f'Pearson: {iter_test.score()}')

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set. ;)
       row_id    target
18211     8_1  0.018524
18212     8_2  0.075989
18213     8_6  0.122681
18214     8_7 -0.062744
18215     8_8  0.002537
...       ...       ...
19995  8_2961 -0.119019
19996  8_2963  0.004261
19997  8_2964 -0.448730
19998  8_2965 -0.013519
19999  8_2966 -0.126831

[1789 rows x 2 columns]
Pearson: (       row_id    target  prediction
0         0_1 -0.300875    0.103760
1         0_2 -0.231040   -0.076843
2         0_6  0.568807    0.052124
3         0_7 -1.064780   -0.157593
4         0_8 -0.531940    0.002863
...       ...       ...         ...
19995  8_2961  2.395875   -0.119019
19996  8_2963  0.293830    0.004261
19997  8_2964  0.295464   -0.448730
19998  8_2965 -0.722136   -0.013519
19999  8_2966 -0.268326   -0.126831

[20000 rows x 3 columns], 0.20382864275050025)


In [9]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df.loc[:, 'f_0':'f_299'], test_df["investment_id"])
    sample_prediction_df['target'] = inference(models, ds)
    env.predict(sample_prediction_df)
print(sample_prediction_df)
print(f'Pearson: {scipy.stats.pearsonr(sample_prediction_df["target"].values, test_df["target"].values)[0]}')

ModuleNotFoundError: No module named 'ubiquant'